# Approach Evaluation

In this notebook, we evaluate the performance of our approach.

In [ ]:
import rich.pretty

rich.pretty.install()

In [ ]:
import IPython.display

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import sklearn.metrics
from sklearn.metrics import RocCurveDisplay
import sqlalchemy as sa
from sklearn.metrics import roc_curve
import msgspec
import itertools
import pathlib as pl
import networkx as nx
import tqdm

In [ ]:
from evaluatie import models as m
from evaluatie import utils
from evaluatie.data import FunctionDataset, DatasetOptions

In [ ]:
tqdm.tqdm.pandas()

In [ ]:
mpl.rc(
    "font",
    size=12,
)

In [ ]:
def create_table(dataset: FunctionDataset, score_col: str, all: bool = True) -> pd.DataFrame:
    categories = [
        "low",
        "medium",
        "high",
    ]
    if all:
        categories.append("all")

    tbl = pd.DataFrame(
        index=pd.Index(
            categories,
            name="neighborhood_size",
        ),
        columns=pd.Index(
            categories,
            name="size",
        ),
        dtype=np.float64,
    )

    for size, neighborhood_size in itertools.product(categories, categories):
        options = DatasetOptions(
            size=size,
            neighborhood_size=neighborhood_size,
        )
        subset_df = dataset.frame[options.indexer(dataset.frame)]

        fpr, tpr, _ = roc_curve(
            y_score=subset_df[score_col],
            y_true=subset_df["label"],
        )

        tbl.loc[neighborhood_size, size] = sklearn.metrics.auc(fpr, tpr)

    return tbl

## BSim

In [ ]:
d = FunctionDataset.from_name("f:o0Xo2")
create_table(d, score_col="bsim")

In [ ]:
d = FunctionDataset.from_name("f:o0Xo3")
create_table(d, score_col="bsim")

In [ ]:
d = FunctionDataset.from_name("f:osXo0")
create_table(d, score_col="bsim")

In [ ]:
d = FunctionDataset.from_name("f:osXo2")
create_table(d, score_col="bsim")

In [ ]:
d = FunctionDataset.from_name("f:noinlineXinline")
create_table(d, score_col="bsim")

In [ ]:
d = FunctionDataset.from_name("f:x86Xarm")
create_table(d, score_col="bsim")

In [ ]:
d = FunctionDataset.from_name("f:armXmips")
create_table(d, score_col="bsim")

In [ ]:
d = FunctionDataset.from_name("f:x86Xmips")
create_table(d, score_col="bsim")

In [ ]:
d = FunctionDataset.from_name("f:malware-analysis")
create_table(d, score_col="bsim")

In [ ]:
d = FunctionDataset.from_name("f:firmware-analysis")
create_table(d, score_col="bsim")

In [ ]:
d = FunctionDataset.from_name("f:random")
create_table(d, score_col="bsim")

## NeighBSim Evaluation [f:]

In [ ]:
name2dataset: dict[str, FunctionDataset] = {}

In [ ]:
# Takes 5 minutes for all datasets
names = [
    # Optimisation
    # "f:o0Xo2",
    # "f:o0Xo2-overview",
    # "f:o0Xo3",
    # "f:o0Xo3-overview",
    # "f:osXo0",
    # "f:osXo0-overview",
    # "f:osXo2",
    # "f:osXo2-overview",
    # "f:osXo3",
    # "f:osXo3-overview",
    # Architecture
    # "f:armXmips",
    # "f:armXmips-overview",
    # "f:x86Xarm",
    # "f:x86Xarm-overview",
    # "f:x86Xmips",
    # "f:x86Xmips-overview",
    # Misc
    # "f:random",
    # "f:random-overview",
    # "f:nopieXpie",
    # "f:nopieXpie-overview",
    # "f:noltoXlto",
    # "f:noltoXlto-overview",
    # "f:noinlineXinline",
    "f:noinlineXinline-overview",
]

for name in names:
    print(f"Loading {name}")
    if name in name2dataset:
        continue
    d = FunctionDataset.from_name(name)
    d = d.load_pickle()
    d = d.drop_metadata(
        keep=[
            "qsize",
            "qneighborhood_size",
            "tsize",
            "tneighborhood_size",
            # Required for testing Assumption 1.
            # Comment out otherwise to save memory
            "caller_matching",
            "callee_matching",
            "qcallers",
            "tcallers",
            "qcallees",
            "tcallees",
        ],
    )

    assert d.frame["bsim"].isna().sum() == 0

    name2dataset[name] = d

In [ ]:
for name, d in name2dataset.items():
    assert len(d.frame[d.frame["neighbsim"].isna()]) == 0, f"{name} has NaN's"
    # d.frame = d.frame.dropna()

In [ ]:
name2label = {
    "f:x86Xarm": "x86 vs. arm",
    "f:x86Xarm-overview": "x86 vs. arm",
    "f:x86Xmips": "x86 vs. mips",
    "f:x86Xmips-overview": "x86 vs. mips",
    "f:armXmips": "arm vs. mips",
    "f:armXmips-overview": "arm vs. mips",
    "f:o0Xo2": "O0 vs. O2",
    "f:o0Xo2-overview": "O0 vs. O2",
    "f:o0Xo3": "O0 vs. O3",
    "f:o0Xo3-overview": "O0 vs. O3",
    "f:osXo0": "Os vs. O0",
    "f:osXo0-overview": "Os vs. O0",
    "f:osXo2": "Os vs. O2",
    "f:osXo2-overview": "Os vs. O2",
    "f:osXo3": "Os vs. O3",
    "f:osXo3-overview": "Os vs. O3",
    "f:noinlineXinline": "noinline vs. inline",
    "f:noinlineXinline-overview": "noinline vs. inline",
    "f:noltoXlto": "LTO",
    "f:noltoXlto-overview": "LTO",
    "f:nopieXpie": "PIE",
    "f:nopieXpie-overview": "PIE",
    "f:random": "Random",
    "f:random-overview": "Random",
}

### Distributions of Sampled Data
This can be used to explain several pehonmenons that we see in our results.

In [ ]:
d = FunctionDataset.from_name("f:armXmips")
d = d.load_pickle()

In [ ]:
df = d.frame.copy()

In [ ]:
df["qneighbor"] = df["qcallees"].apply(len) + df["qcallers"].apply(len)
df["tneighbor"] = df["tcallees"].apply(len) + df["tcallers"].apply(len)

In [ ]:
sns.histplot(
    data=df[(df["qneighbor"] < 50) & (df["qneighborhood_size"] == "high")],
    x="qneighbor",
)

### When BSim outperforms NeighBSim

In [ ]:
for name, d in name2dataset.items():
    print()
    print(name)
    bsim = (
        create_table(
            d,
            score_col="bsim",
            all=False,
        )
        .stack()
        .reset_index(
            name="score",
        )
    )

    neighbsim = (
        create_table(
            d,
            score_col="neighbsim",
            all=False,
        )
        .stack()
        .reset_index(
            name="score",
        )
    )

    print(bsim[bsim["score"] > neighbsim["score"]])

In [ ]:
bsim

In [ ]:
d = FunctionDataset.from_name("f:o0Xo2")
d = d.load_pickle()

In [ ]:
opts = DatasetOptions(
    size="high",
    neighborhood_size="high",
)

In [ ]:
df = d.frame[opts.indexer(d.frame)].copy()

In [ ]:
df = df[(df["label"] == False) & (df["neighbsim"] > 0.4)]

In [ ]:
def avg_edge_weight(graph: nx.Graph) -> float | None:
    if len(graph.edges) == 0:
        return None

    return float(
        np.average(
            [weight for _, _, weight in graph.edges(data="weight")],
        )
    )

In [ ]:
df["avg_callee_edge"] = df["callee_matching"].apply(avg_edge_weight)

In [ ]:
sns.histplot(
    data=df,
    x="avg_callee_edge",
)

### Score-Distributions

In [ ]:
import seaborn as sns

In [ ]:
score_col2label = {
    "bsim": "BSim",
    "neighbsim": "NeighBSim",
}

In [ ]:
def plot_kde(
    d: FunctionDataset,
    score_col: str,
    ax: mpl.axis.Axis,
    dataset_options: DatasetOptions | None = None,
):
    plot_df = d.frame.copy()
    if dataset_options is not None:
        plot_df = plot_df.loc[dataset_options.indexer(plot_df)]

    plot_df["label"] = plot_df["label"].map(
        {
            True: "Positive",
            False: "Negative",
        },
    )

    sns.kdeplot(
        data=plot_df,
        x=score_col,
        hue="label",
        cut=0,
        clip=(0, 1),
        fill=True,
        common_norm=False,
        ax=ax,
    )

    ax.set_xlim(0, 1)
    ax.set_xlabel(score_col2label[score_col])

In [ ]:
def create_score_distribution_subplots(*, name: str, score_col: str):
    fig, axs = plt.subplots(
        nrows=3,
        ncols=3,
        # Turned off to set labels for each axis individually
        sharex=False,
        sharey=True,
    )

    def opts_from_idx(*, row, col):
        idx2bin = {
            0: "low",
            1: "medium",
            2: "high",
        }

        return DatasetOptions(
            size=idx2bin[col],
            neighborhood_size=idx2bin[row],
        )

    for row, col_axs in enumerate(axs):
        for col, ax in enumerate(col_axs):
            opts = opts_from_idx(
                row=row,
                col=col,
            )
            plot_kde(
                name2dataset[name],
                score_col=score_col,
                ax=ax,
                dataset_options=opts,
            )

            ax.set_xticklabels([])
            ax.set_xlabel("Score")
            ax.set_ylabel("Density")

            ax.yaxis.set_label_position("right")
            # Needed to make the label show up on the rightmost plot, not on the
            # leftmost
            ax.yaxis.tick_right()
            ax.tick_params(axis="y", labelright=True)

    # Somehow we cannot call this ins the above loop
    for ax in axs.flatten():
        ax.label_outer()

    axs[0][0].get_legend().remove()
    axs[0][1].get_legend().remove()
    # axs[0][2].get_legend().remove()
    axs[1][0].get_legend().remove()
    axs[1][1].get_legend().remove()
    axs[1][2].get_legend().remove()
    axs[2][0].get_legend().remove()
    axs[2][1].get_legend().remove()
    axs[2][2].get_legend().remove()

    axs[0][2].get_legend().set_title("Label")

    axs[0][0].set_title(
        "Low",
        loc="center",
    )
    axs[0][1].set_title(
        "Medium",
        loc="center",
    )
    axs[0][2].set_title(
        "High",
        loc="center",
    )

    axs[0][0].set_title(
        "Low",
        loc="left",
        y=0.5,
        rotation="vertical",
        va="center",
        x=-0.15,
    )
    axs[1][0].set_title(
        "Medium",
        loc="left",
        y=0.5,
        rotation="vertical",
        va="center",
        x=-0.15,
    )
    axs[2][0].set_title(
        "High",
        loc="left",
        y=0.5,
        rotation="vertical",
        va="center",
        x=-0.15,
    )

    xticks = [0, 0.5, 1]
    axs[2][0].set_xticks(xticks)
    axs[2][1].set_xticks(xticks)
    axs[2][2].set_xticks(xticks)
    axs[2][0].set_xticklabels([0.0, 0.5, 1.0])
    axs[2][1].set_xticklabels(["", 0.5, ""])
    axs[2][2].set_xticklabels([0.0, 0.5, 1.0])

    axs[0][0].set_yticks([0, 10, 20])
    axs[0][0].set_yticklabels([0, 10, ""])
    axs[0][0].set_ylim(0, 20)

    fig.subplots_adjust(
        wspace=0,
        hspace=0,
    )

    fig.suptitle(
        "#BasicBlocks",
    )
    fig.supylabel(
        "#Neighbors",
        x=0.03,
    )

    return fig, axs

In [ ]:
score_col = "bsim"

for name in [
    # Optimisation
    "f:o0Xo2",
    "f:o0Xo3",
    "f:osXo0",
    "f:osXo2",
    "f:osXo3",
    # Architecture
    "f:armXmips",
    "f:x86Xarm",
    "f:x86Xmips",
    # Misc
    "f:noinlineXinline",
    "f:nopieXpie",
    "f:noltoXlto",
    "f:random",
]:
    score_col = "bsim"
    fig, axs = create_score_distribution_subplots(
        name=name,
        score_col=score_col,
    )
    fig.savefig(f"figures/score-dist/{name}-{score_col}.pdf")

    score_col = "neighbsim"
    fig, axs = create_score_distribution_subplots(
        name=name,
        score_col=score_col,
    )
    fig.savefig(f"figures/score-dist/{name}-{score_col}.pdf")

### (Neigh)BSim ROC Curves

In [ ]:
import sklearn
import sklearn.metrics

In [ ]:
def roc_curve_from_dataset(
    d: FunctionDataset,
    score_col: str,
    opts: DatasetOptions | None = None,
):
    frame = d.frame
    if opts is not None:
        frame = d.frame[opts.indexer(frame)]

    fpr, tpr, _ = sklearn.metrics.roc_curve(
        y_true=frame["label"],
        y_score=frame[score_col],
    )

    return fpr, tpr

In [ ]:
def plot_roc(fpr, tpr, *, ax):
    lines = ax.plot(
        fpr,
        tpr,
        drawstyle="steps-post",
        clip_on=False,
    )
    assert len(lines) == 1
    line = lines[0]

    ax.set_xlabel("False-Positive Rate")
    ax.set_ylabel("True-Positive Rate")

    ax.set_aspect("equal")
    ax.set_xlim(
        xmin=0,
        xmax=1,
    )
    ax.set_ylim(
        ymin=0,
        ymax=1,
    )

    return line


def plot_roc_random(ax):
    ax.plot(
        [0, 1],
        [0, 1],
        "--",
        color="gray",
        label="Random",
    )

In [ ]:
d = name2dataset["f:armXmips"]
e = name2dataset["f:o0Xo3"]

In [ ]:
d_fpr, d_tpr = roc_curve_from_dataset(d, score_col="neighbsim")
sklearn.metrics.auc(d_fpr, d_tpr)

In [ ]:
e_fpr, e_tpr = roc_curve_from_dataset(e, score_col="bsim")
sklearn.metrics.auc(e_fpr, e_tpr)

In [ ]:
fig, ax_roc = plt.subplots()

line = plot_roc(
    d_fpr,
    d_tpr,
    ax=ax_roc,
)
line.set_label("ARM vs. MIPS (NeighBSim)")

line = plot_roc(
    e_fpr,
    e_tpr,
    ax=ax_roc,
)
line.set_label("O0 vs. O3 (BSim)")

plot_roc_random(
    ax=ax_roc,
)

ax_roc.legend()

In [ ]:
fig.savefig(
    "figures/roc.pdf",
    bbox_inches="tight",
    transparent=True,
)

### Overview Bar Chart

In [ ]:
import seaborn as sns
import matplotlib.axis
import matplotlib as mpl
import matplotlib.cm

In [ ]:
def auc_from_dataset(d: FunctionDataset, score_col: str):
    return sklearn.metrics.roc_auc_score(
        y_true=d.frame["label"],
        y_score=d.frame[score_col],
    )

In [ ]:
values = []
for d in name2dataset.values():
    bsim_auc = auc_from_dataset(d, "bsim")
    neighbsim_auc = auc_from_dataset(d, "neighbsim")

    values.append((d.name, bsim_auc, "bsim"))
    values.append((d.name, neighbsim_auc, "neighbsim"))

df = pd.DataFrame(
    data=values,
    columns=[
        "name",
        "auc",
        "score",
    ],
)


df = pd.pivot(
    df,
    index="name",
    columns="score",
    values="auc",
)

df = df.loc[
    [
        "f:o0Xo2-overview",
        "f:o0Xo3-overview",
        "f:osXo0-overview",
        "f:osXo2-overview",
        "f:osXo3-overview",
        "f:x86Xarm-overview",
        "f:x86Xmips-overview",
        "f:armXmips-overview",
        "f:noinlineXinline-overview",
        "f:noltoXlto-overview",
        "f:nopieXpie-overview",
        "f:random-overview",
    ]
]

In [ ]:
def format_auc(value):
    return f"{value:.3f}".lstrip("0")


fig, ax_roc = plt.subplots(
    figsize=(12, 4),
)

width = 0.35
x = np.arange(len(df))

rects = ax_roc.bar(
    x=x,
    height=df["bsim"],
    label="BSim",
    width=width,
    color="green",
)
ax_roc.bar_label(
    rects,
    padding=3,
    fmt=format_auc,
    rotation=0,
    fontsize="x-small",
)

rects = ax_roc.bar(
    x=x + width,
    height=df["neighbsim"],
    width=width,
    label="NeighBSim",
    # color="crimson",
    color="tomato",
)
ax_roc.bar_label(
    rects,
    padding=3,
    fmt=format_auc,
    rotation=0,
    fontsize="x-small",
)

ax_roc.set_ylim((0.9, 1.0))
ax_roc.set_ylabel("AUC")
ax_roc.legend(
    loc="lower left",
)

ax_roc.set_yticks(np.arange(0.90, 1.01, 0.01))


_ = ax_roc.set_xticks(
    x + 0.5 * width,
    labels=df.index.to_series().apply(lambda name: name2label[name]),
    rotation=60,
    fontsize="medium",
)

ax_roc.grid(
    color="grey",
    linewidth=0.4,
    axis="y",
)

In [ ]:
fig.savefig(
    "figures/evaluation:neighbsim-vs-bsim-barchart.pdf",
    bbox_inches="tight",
    transparent=True,
)

### Heatmaps 

In [ ]:
import seaborn as sns
import matplotlib.axis
import matplotlib as mpl
import matplotlib.cm

In [ ]:
def setup_axis(ax):
    # ax.invert_yaxis()
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position("top")

    labels = [
        "Low",
        "Medium",
        "High",
    ]
    ax.set_xticks(
        ticks=np.arange(len(labels)),
        labels=labels,
    )
    ax.set_yticks(
        ticks=np.arange(len(labels)),
        labels=labels,
    )

    ax.set_aspect(
        "equal",
    )

    ax.set_xticks(
        np.arange(len(labels)) - 0.5,
        minor=True,
    )
    ax.set_yticks(
        np.arange(len(labels)) - 0.5,
        minor=True,
    )
    ax.grid(
        visible=True,
        color="black",
        which="minor",
    )

    ax.tick_params(which="minor", bottom=False, left=False, top=False, right=False)

    ax.set_xlabel("#BasicBlocks")
    ax.set_ylabel("#Neighbors")

In [ ]:
def plot_heatmap(
    d: FunctionDataset,
    score_col: str,
    ax,
):
    setup_axis(ax)

    tbl = create_table(
        d,
        score_col=score_col,
        all=False,
    )

    im = ax.imshow(
        tbl.to_numpy(),
        cmap="YlGn",
        vmin=0.9,
        vmax=1.0,
    )

    for i in range(len(tbl.columns)):
        for j in range(len(tbl)):
            auc = tbl.iloc[j, i]
            auc_str = "{0:.3f}".format(auc).lstrip("0")
            ax.text(
                i,
                j,
                auc_str,
                ha="center",
                va="center",
                color="black",
            )

    return im

In [ ]:
def child_by_type(ax, ty):
    children = [child for child in ax.get_children() if isinstance(child, ty)]
    if len(children) > 1:
        raise ValueError(f"Multiple children of type {ty}")

    if len(children) == 0:
        raise ValueError(f"No children of type {ty}")

    return children[0]

In [ ]:
def create_heatmap_figure(dataset_names: list[str], score_col: str):
    size_per_dataset = 3
    fig, axs = plt.subplots(
        nrows=1,
        ncols=len(dataset_names),
        sharey=True,
        figsize=(size_per_dataset * len(dataset_names), 3),
    )

    for name, ax_roc in zip(
        dataset_names,
        axs,
    ):
        d = name2dataset[name]
        plot_heatmap(
            d,
            score_col=score_col,
            ax=ax_roc,
        )

        ax_roc.set_title(
            name2label[d.name],
            y=0,
            pad=-20,
        )

    for ax_roc in axs[1:]:
        ax_roc.tick_params(
            axis="y",
            left=False,
            labelleft=False,
            which="major",
        )
        ax_roc.set_ylabel(None)

    im = child_by_type(axs[0], matplotlib.image.AxesImage)
    cbar = fig.colorbar(im, ax=axs)

    return fig

In [ ]:
fig = create_heatmap_figure(
    dataset_names=[
        "f:o0Xo2",
        "f:o0Xo3",
        "f:osXo0",
        "f:osXo2",
        "f:osXo3",
    ],
    score_col="neighbsim",
)

fig.savefig(
    "figures/heatmap-optimization-neighbsim.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
fig = create_heatmap_figure(
    dataset_names=[
        "f:o0Xo2",
        "f:o0Xo3",
        "f:osXo0",
        "f:osXo2",
        "f:osXo3",
    ],
    score_col="bsim",
)

fig.savefig(
    "figures/heatmap-optimization-bsim.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
fig = create_heatmap_figure(
    dataset_names=[
        "f:x86Xarm",
        "f:x86Xmips",
        "f:armXmips",
    ],
    score_col="neighbsim",
)

fig.savefig(
    "figures/heatmap-architecture-neighbsim.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
fig = create_heatmap_figure(
    dataset_names=[
        "f:x86Xarm",
        "f:x86Xmips",
        "f:armXmips",
    ],
    score_col="bsim",
)

fig.savefig(
    "figures/heatmap-architecture-bsim.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
fig = create_heatmap_figure(
    dataset_names=[
        "f:noinlineXinline",
        "f:noltoXlto",
        "f:nopieXpie",
        "f:random",
    ],
    score_col="neighbsim",
)

fig.savefig(
    "figures/heatmap-misc-neighbsim.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
fig = create_heatmap_figure(
    dataset_names=[
        "f:noinlineXinline",
        "f:noltoXlto",
        "f:nopieXpie",
        "f:random",
    ],
    score_col="bsim",
)

fig.savefig(
    "figures/heatmap-misc-bsim.pdf",
    bbox_inches="tight",
    transparent=True,
)

#### Other

In [ ]:
for name, d in name2dataset.items():
    neighbsim_table = create_table(d, score_col="neighbsim")
    IPython.display.display(f"{name} -- NeighBSim")
    IPython.display.display(neighbsim_table)

    bsim_table = create_table(d, score_col="bsim")
    IPython.display.display(f"{name} -- BSim")
    IPython.display.display(bsim_table)

In [ ]:
bins = ["low", "medium", "high"]

x = np.arange(len(bins) * len(name2dataset), step=len(name2dataset))
inter_dataset_offset = 0.4
intra_dataset_offset = 0.2

In [ ]:
markers = ["o", "x", "1", "<", "D"]

In [ ]:
bin2line = {
    "low": "dotted",
    "medium": (0, (3, 1, 1, 1)),
    "high": "solid",
}

In [ ]:
fig, ax_roc = plt.subplots(
    figsize=(8, 12),
)

In [ ]:
marker_cycle = iter(markers)

for i, dataset in enumerate(name2dataset.values()):
    marker = next(marker_cycle)
    bsim_table = create_table(dataset, "bsim").drop(columns="all").drop(labels="all")
    neighbsim_table = create_table(dataset, "neighbsim").drop(columns="all").drop(labels="all")

    dataset_offset = i * (inter_dataset_offset + 2 * intra_dataset_offset)

    for j, size_bin in enumerate(bins):
        offset = dataset_offset + j * intra_dataset_offset

        bsim_auc = bsim_table[size_bin]
        neighbsim_auc = neighbsim_table[size_bin]

        ymin = np.where(bsim_auc < neighbsim_auc, bsim_auc, neighbsim_auc)
        ymax = np.where(bsim_auc > neighbsim_auc, bsim_auc, neighbsim_auc)
        ax_roc.vlines(
            x + offset, ymin=ymin, ymax=ymax, colors="grey", linestyles=bin2line[size_bin]
        )

        rects = ax_roc.scatter(
            x=x + offset,
            y=bsim_auc,
            # width=width,
            label=size_bin,
            color="mediumseagreen",
            alpha=1.0,
            marker=marker,
        )

        rects = ax_roc.scatter(
            x=x + offset,
            y=neighbsim_auc,
            # width=width,
            label=size_bin,
            color="tomato",
            alpha=1.0,
            marker=marker,
        )

### Additional Plots
As the logic above is quite generic, we use it to create all sorts of plots that we use in the thesis.

#### Background & Definitions: ROC Introduction

In [ ]:
opts = DatasetOptions(
    size="high",
    neighborhood_size="low",
)

left_dataset = name2dataset["f:noinlineXinline"]
right_dataset = name2dataset["f:o0Xo2"]

In [ ]:
fig, (ax_left, ax_right, ax_roc) = plt.subplots(nrows=1, ncols=3, figsize=(14, 3))

# Distributions
plot_kde(
    left_dataset,
    score_col="bsim",
    ax=ax_left,
    dataset_options=opts,
)

plot_kde(
    right_dataset,
    score_col="bsim",
    ax=ax_right,
    dataset_options=opts,
)

# thresh = 0.6
# height = ax_left.get_ylim()[1]
# ax_left.vlines(
#    thresh,
#    ymin=0,
#    ymax=height,
#    colors="grey",
#    linestyles="dashed",
# )

# thresh = 0.2
# height = ax_right.get_ylim()[1]
# ax_right.vlines(
#    thresh,
#    ymin=0,
#    ymax=height,
#    colors="grey",
#    linestyles="dashed",
# )

ax_left.get_legend().set_loc("upper left")


# ROC Plot
ltpr, lfpr = roc_curve_from_dataset(left_dataset, score_col="bsim", opts=opts)
line = plot_roc(ltpr, lfpr, ax=ax_roc)
line.set_color("olivedrab")
line.set_label("Left")


rtpr, rfpr = roc_curve_from_dataset(right_dataset, score_col="bsim", opts=opts)
line = plot_roc(rtpr, rfpr, ax=ax_roc)
line.set_color("firebrick")
line.set_label("Right")

plot_roc_random(
    ax=ax_roc,
)

ax_left.get_legend().set_title("Label")
ax_right.get_legend().set_title("Label")

ax_roc.set_xticks(ax_roc.get_yticks())


_ = ax_roc.legend()

In [ ]:
fig.savefig(
    "figures/basics:roc-example.pdf",
    bbox_inches="tight",
    transparent=True,
)

### Does _Assumption 1_ hold, and how does maximum weight matching perform?

In [ ]:
import seaborn as sns
import matplotlib.patches
import matplotlib.pyplot as plt

In [ ]:
colorblind_palette = sns.color_palette("colorblind")
green = colorblind_palette[2]
red = colorblind_palette[3]

In [ ]:
id2name: dict = {}


def populate_id2name():
    ids = []
    for d in name2dataset.values():
        _ = d.frame["callee_matching"].progress_apply(lambda matching: ids.extend(matching.nodes))
        _ = d.frame["caller_matching"].progress_apply(lambda matching: ids.extend(matching.nodes))

    with m.Session() as session:
        stmt = sa.select(
            m.Function.id,
            m.Function.name,
        ).where(m.Function.id.in_(ids))

        for id, name in session.execute(stmt):
            id2name[id] = name

In [ ]:
# Takes roughly 30 seconds for two datasets
populate_id2name()

In [ ]:
def matching_is_correct(matching: nx.Graph):
    # Do not count empty matchings as correct
    if len(matching.edges) == 0:
        return None

    ret = True
    for src_id, dst_id in matching.edges:
        ret &= id2name[src_id] == id2name[dst_id]

    return ret

In [ ]:
def _function_sets_are_equivalent(left: list, right: list):
    if len(left) != len(right):
        return False

    left_names = {id2name[id] for id in left}
    right_names = {id2name[id] for id in right}

    return left_names == right_names


def callees_changed(row):
    qcallees = row["qcallees"]
    tcallees = row["tcallees"]

    return not _function_sets_are_equivalent(qcallees, tcallees)


def callers_changed(row):
    qcallers = row["qcallers"]
    tcallers = row["tcallers"]

    return not _function_sets_are_equivalent(qcallers, tcallers)

In [ ]:
def plot_frame_from_dataset(d: FunctionDataset):
    plot_df = d.frame[
        # We are only interested in functions labeled as positive.
        # For negative functions it is expected that the matching is wrong and the set of callers/callees changed
        (d.frame["label"] == True)
        &
        # Drop all columns that where either side of the bipartite graph is empty
        (d.frame["caller_matching"].apply(lambda matching: len(matching.edges)) != 0)
        &
        # Same for the callees
        (d.frame["callee_matching"].apply(lambda matching: len(matching.edges)) != 0)
    ].copy()

    plot_df["callees_changed"] = d.frame.apply(callees_changed, axis=1)
    plot_df["callers_changed"] = d.frame.apply(callers_changed, axis=1)
    plot_df["callee_matching_correct"] = d.frame["callee_matching"].apply(matching_is_correct)
    plot_df["caller_matching_correct"] = d.frame["caller_matching"].apply(matching_is_correct)

    return plot_df

In [ ]:
def data_from_plot_frame(plot_df: pd.DataFrame):
    count = len(plot_df)
    return {
        "pair-count": count,
        "changed-callers": plot_df["callers_changed"].sum() / count,
        "changed-callees": plot_df["callees_changed"].sum() / count,
        "caller-matching-correct": plot_df["caller_matching_correct"].sum() / count,
        "callee-matching-correct": plot_df["callee_matching_correct"].sum() / count,
        # Callers
        "callee-matching-correct-changed": plot_df[plot_df["callees_changed"] == True][
            "callee_matching_correct"
        ].sum()
        / count,
        "callee-matching-correct-unchanged": plot_df[plot_df["callees_changed"] == False][
            "callee_matching_correct"
        ].sum()
        / count,
        # Callees
        "caller-matching-correct-changed": plot_df[plot_df["callers_changed"] == True][
            "caller_matching_correct"
        ].sum()
        / count,
        "caller-matching-correct-unchanged": plot_df[plot_df["callers_changed"] == False][
            "caller_matching_correct"
        ].sum()
        / count,
    }

In [ ]:
import pathlib as pl
import pickle

with pl.Path("datasets/neighborhood-correctness.pickle").open("rb") as f:
    name2data = pickle.load(f)

# name2data = {}

In [ ]:
name2data = {
    # Optimization
    "f:o0Xo2-overview": name2data["f:o0Xo2-overview"],
    "f:o0Xo3-overview": name2data["f:o0Xo3-overview"],
    "f:osXo0-overview": name2data["f:osXo0-overview"],
    "f:osXo2-overview": name2data["f:osXo2-overview"],
    "f:osXo3-overview": name2data["f:osXo3-overview"],
    # Architecture
    "f:x86Xarm": name2data["f:x86Xarm"],
    "f:x86Xmips": name2data["f:x86Xmips"],
    "f:armXmips": name2data["f:armXmips"],
    # Misc
    "f:noinlineXinline-overview": name2data["f:noinlineXinline-overview"],
    "f:noltoXlto-overview": name2data["f:noltoXlto-overview"],
    "f:nopieXpie-overview": name2data["f:nopieXpie-overview"],
    "f:random-overview": name2data["f:random-overview"],
}

In [ ]:
def plot_neighborhood_changes(caller_or_callee: str):
    fig, ax = plt.subplots(
        figsize=(12, 4),
    )

    for name, d in name2dataset.items():
        if name in name2data:
            continue
        plot_df = plot_frame_from_dataset(d)
        name2data[name] = data_from_plot_frame(plot_df)

    width = 0.5
    x = list(name2data.keys())

    changed_callers = np.array(
        [data[f"changed-{caller_or_callee}s"] for data in name2data.values()]
    )
    unchanged_callers = np.array([1 - v for v in changed_callers])

    caller_matching_correct_changed = np.array(
        [data[f"{caller_or_callee}-matching-correct-changed"] for data in name2data.values()]
    )
    caller_matching_correct_unchanged = np.array(
        [data[f"{caller_or_callee}-matching-correct-unchanged"] for data in name2data.values()]
    )

    # Changed
    ax.bar(
        x=x,
        bottom=0,
        height=caller_matching_correct_changed,
        width=width,
        label="Changed",
        color=red,
    )
    ax.bar(
        x=x,
        bottom=caller_matching_correct_changed,
        height=changed_callers - caller_matching_correct_changed,
        width=width,
        label="Changed",
        color=red,
        alpha=0.7,
    )

    # Unchanged
    ax.bar(
        x=x,
        bottom=1,
        height=-caller_matching_correct_unchanged,
        width=width,
        label="Unchanged",
        color=green,
    )
    ax.bar(
        x=x,
        bottom=1 - caller_matching_correct_unchanged,
        height=-(unchanged_callers - caller_matching_correct_unchanged),
        width=width,
        label="Unchanged",
        color=green,
        alpha=0.7,
    )

    # Setup axes
    ax.set_ylim(0, 1)
    ax.set_yticks(np.arange(0, 1.1, 0.1))
    xticks = ax.get_xticks()
    ax.set_xticks(
        xticks,
        labels=[name2label[label.get_text()] for label in ax.get_xticklabels()],
        rotation=60,
    )

    ax.grid(
        color="grey",
        linewidth=0.4,
        axis="y",
    )
    ax.set_ylabel("Percentage")

    rect_correct = matplotlib.patches.Patch(
        edgecolor=green,
        facecolor=green + (0.7,),
        linewidth=3,
        label="Unchanged",
    )
    rect_incorrect = matplotlib.patches.Patch(
        edgecolor=red,
        facecolor=red + (0.7,),
        linewidth=3,
        label="Changed",
    )

    ax.legend(handles=[rect_correct, rect_incorrect])

    return fig, ax

In [ ]:
fig, ax = plot_neighborhood_changes("callee")

fig.savefig(
    "figures/call-graph-matching-correctness-callee.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
fig, ax = plot_neighborhood_changes("caller")

fig.savefig(
    "figures/call-graph-matching-correctness-caller.pdf",
    bbox_inches="tight",
    transparent=True,
)

### Statistical Significance (DeLong Test)

In [ ]:
from MLstatkit.stats import Delong_test

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pandas as pd

In [ ]:
def p_values_from_dataset(d: FunctionDataset):
    bins = ["low", "medium", "high"]

    ret = pd.Series(
        index=pd.MultiIndex.from_tuples(
            itertools.product(bins, bins),
            names=["size", "neighborhood_size"],
        )
    )

    for size in bins:
        for neighborhood_size in bins:
            opts = DatasetOptions(
                size=size,
                neighborhood_size=neighborhood_size,
            )

            df = d.frame[opts.indexer(d.frame)]

            labels = df.label.apply(int)
            bsim = df.bsim
            neighbsim = df.neighbsim

            z_score, p_val = Delong_test(
                true=labels,
                prob_A=bsim,
                prob_B=neighbsim,
            )

            ret.loc[size, neighborhood_size] = p_val

    return ret

In [ ]:
def p_value_from_dataset(d: FunctionDataset):
    df = d.frame

    labels = df.label.apply(int)
    bsim = df.bsim
    neighbsim = df.neighbsim

    z_score, p_val = Delong_test(
        true=labels,
        prob_A=bsim,
        prob_B=neighbsim,
    )

    return p_val

In [ ]:
bins = ["low", "medium", "high"]
df = pd.DataFrame(
    columns=list({name.removesuffix("-overview") for name in name2dataset}),
    index=pd.MultiIndex.from_tuples(
        itertools.chain(itertools.product(bins, bins), [("all", "all")]),
        names=["size", "neighborhood_size"],
    ),
    dtype=np.float128,
)

for name, d in name2dataset.items():
    if name.endswith("-overview"):
        name = name.removesuffix("-overview")
        df.loc[("all", "all"), name] = p_value_from_dataset(d)
    else:
        ret = p_values_from_dataset(d)
        df.loc[ret.index, name] = ret

df

In [ ]:
df.to_csv("figures/significance.csv")

In [ ]:
df = pd.read_csv("figures/significance.csv").set_index(["size", "neighborhood_size"])

In [ ]:
df = df[
    [
        # Optimization
        "f:o0Xo2",
        "f:o0Xo3",
        "f:osXo0",
        "f:osXo2",
        "f:osXo3",
        # Architecture
        "f:x86Xarm",
        "f:x86Xmips",
        "f:armXmips",
        # Misc
        "f:noinlineXinline",
        "f:noltoXlto",
        "f:nopieXpie",
        "f:random",
    ]
]

In [ ]:
df

In [ ]:
def format_cell(p_value):
    x = f"{p_value:.2E}"
    num, exponent = x.split("E")

    ret = f"${num} \\cdot 10^{{{exponent}}}$"
    if p_value >= 0.01:
        ret = r"\cellcolor{lightgray}" + ret

    return ret

In [ ]:
df.applymap(format_cell)

In [ ]:
def format_row(row, comment):
    row = row.apply(format_cell)

    return "& " + " & ".join(row) + f" % {comment}"

In [ ]:
df[
    [
        # Architecture
        "f:x86Xarm",
        "f:x86Xmips",
        "f:armXmips",
    ]
].apply(format_row, comment="Architecture", axis=1).to_csv("/tmp/df.csv", index=False)

In [ ]:
df[
    [
        # Optimization
        "f:o0Xo2",
        "f:o0Xo3",
        "f:osXo0",
        "f:osXo2",
        "f:osXo3",
    ]
].apply(format_row, comment="Optimization", axis=1).to_csv("/tmp/df.csv", index=False)

In [ ]:
df[
    [
        # Misc
        "f:noinlineXinline",
        "f:noltoXlto",
        "f:nopieXpie",
        "f:random",
    ]
].apply(format_row, comment="Misc", axis=1).to_csv("/tmp/df.csv", index=False)

In [ ]:
df >= 0.01

### Matching Weights vs. Function Score

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns
import pandas as pd

In [ ]:
d = name2dataset["f:o0Xo2-overview"]

In [ ]:
def avg_edge_weight(matching: nx.Graph):
    weights = [data["weight"] for _, _, data in matching.edges(data=True)]

    if len(weights) == 0:
        return np.nan

    return sum(weights) / len(weights)

In [ ]:
def avg_neighborhood_weight(row):
    caller_matching = row["caller_matching"]
    callee_matching = row["callee_matching"]

    caller_avg = avg_edge_weight(caller_matching)
    callee_avg = avg_edge_weight(callee_matching)

    if np.isnan(caller_avg) and np.isnan(callee_avg):
        return np.nan

    if np.isnan(caller_avg):
        return callee_avg

    if np.isnan(callee_avg):
        return caller_avg

    return (caller_avg * len(caller_matching.edges) + callee_avg * len(caller_matching.edges)) / (
        len(caller_matching.edges) + len(callee_matching.edges)
    )

In [ ]:
df = d.frame
df = df[(df["label"] == True) & (df["qneighborhood_size"] == "high")].copy()

In [ ]:
df["avg"] = df.apply(
    avg_neighborhood_weight,
    axis=1,
)

In [ ]:
step = 0.1
bins = np.arange(
    0,
    1 + step,
    step,
)

In [ ]:
df["bsim-bin"] = pd.cut(
    df["bsim"],
    bins=bins,
)

In [ ]:
df["avg-bin"] = pd.cut(
    df["avg"],
    bins=bins,
)

In [ ]:
plot_df = pd.crosstab(index=df["bsim-bin"], columns=df["avg-bin"])

In [ ]:
plot_df

In [ ]:
fig, ax = plt.subplots()

sns.heatmap(
    data=plot_df,
    ax=ax,
)

ax.xaxis.tick_top()
ax.xaxis.set_tick_params(rotation=300)
ax.xaxis.set_label_position("top")

## NeighBSim Evaluation [mrr:]

In [ ]:
import pandas as pd
import seaborn as sns
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [ ]:
names = [
    "mrr:o0Xo2",
    "mrr:o0Xo3",
    "mrr:osXo0",
    "mrr:osXo2",
    "mrr:osXo3",
    "mrr:armXmips",
    "mrr:x86Xarm",
    "mrr:x86Xmips",
    "mrr:noinlineXinline",
    #'mrr:noltoXlto',
    "mrr:nopieXpie",
    #'mrr:randomXrandom',
]

name2frame = {}

for name in names:
    print(name)
    ranks_frame = pd.read_csv(f"datasets/{name}-ranks.csv", index_col=0)
    firmup_frame = pd.read_csv(f"datasets/{name}-firmup.csv", index_col=0)[
        ["firmup", "firmup-steps"]
    ]
    frame = pd.concat([ranks_frame, firmup_frame], axis=1)
    name2frame[name] = frame

In [ ]:
name2label = {
    "mrr:x86Xarm": "x86 vs. arm",
    "mrr:x86Xmips": "x86 vs. mips",
    "mrr:armXmips": "arm vs. mips",
    "mrr:o0Xo2": "O0 vs. O2",
    "mrr:o0Xo3": "O0 vs. O3",
    "mrr:osXo0": "Os vs. O0",
    "mrr:osXo2": "Os vs. O2",
    "mrr:osXo3": "Os vs. O3",
    "mrr:noinlineXinline": "noinline vs. inline",
    "mrr:noltoXlto": "LTO",
    "mrr:nopieXpie": "PIE",
    "mrr:random": "Random",
}

### Rank Distributions

In [ ]:
def plot_rank_distribution(frame: pd.DataFrame, *, score_col: str, ax):
    plot_df = frame.copy()

    sns.histplot(
        data=plot_df,
        x=score_col,
        discrete=True,
        stat="probability",
        label=score_col,
        alpha=0.7,
        ax=ax,
    )

In [ ]:
def plot_rank_distribution2(frame: pd.DataFrame, *, ax):
    plot_df = frame.copy()

    plot_df = pd.melt(
        plot_df,
        id_vars=[
            "query_binary_id",
            "target_binary_id",
            "query_function_id",
            "ptarget_function_id",
        ],
        value_vars=["bsim_rank", "neighbsim_rank"],
        value_name="rank",
        var_name="score_col",
    )

    sns.histplot(
        data=plot_df,
        x="rank",
        hue="score_col",
        discrete=True,
        stat="probability",
        alpha=0.7,
        ax=ax,
    )

In [ ]:
def plot_rank_distribution_cumul(frame: pd.DataFrame, *, score_col: str, ax):
    plot_df = frame.copy()

    sns.histplot(
        data=plot_df,
        x=score_col,
        discrete=True,
        stat="probability",
        element="step",
        cumulative=True,
        fill=False,
        # label=score_col,
        alpha=1.0,
        ax=ax,
    )

In [ ]:
names = [
    "mrr:o0Xo2",
    "mrr:o0Xo3",
    "mrr:osXo0",
    "mrr:osXo2",
    "mrr:osXo3",
]

fig, axs = plt.subplots(
    nrows=1,
    ncols=len(names),
    figsize=(15, 2.5),
    sharex=True,
    sharey=True,
)


max_rank = 10


for ax in axs:
    ax.set_xlim((0, max_rank))
    ax.set_ylim(0, 1)

    ax.set_xticks(np.arange(1, 11))
    ax.set_yticks(np.arange(0, 1.1, 0.2))

axs[0].set_ylabel("Percentage")

for (
    ax,
    name,
) in zip(axs, names):
    frame = name2frame[name]
    plot_rank_distribution_cumul(
        frame,
        score_col="bsim_rank",
        ax=ax,
    )

    plot_rank_distribution_cumul(
        frame,
        score_col="neighbsim_rank",
        ax=ax,
    )

    ax.set_xlabel("Rank")


for (
    ax,
    name,
) in zip(axs, names):
    frame = name2frame[name]
    plot_rank_distribution(
        frame,
        score_col="bsim_rank",
        ax=ax,
    )

    plot_rank_distribution(
        frame,
        score_col="neighbsim_rank",
        ax=ax,
    )

    ax.set_title(
        name2label[name],
    )
# Omit legend here to only show it in the smaller architecture graph.
# legend = axs[-1].legend(
#     loc="upper left",
#     bbox_to_anchor=(1.02, 1),
# )
# score_col2text = {
#     "bsim_rank": "BSim",
#     "neighbsim_rank": "NeighBSim",
# }
# for text in legend.get_texts():
#     text.set_text(score_col2text[text.get_text()])

In [ ]:
fig.savefig(
    "figures/ranking:optimization.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
names = [
    "mrr:armXmips",
    "mrr:x86Xarm",
    "mrr:x86Xmips",
]

fig, axs = plt.subplots(
    nrows=1,
    ncols=len(names),
    figsize=(3 * len(names), 2.5),
    sharex=True,
    sharey=True,
)


max_rank = 10


for ax in axs:
    ax.set_xlim((0, max_rank))
    ax.set_ylim(0, 1)

    ax.set_xticks(np.arange(1, 11))
    ax.set_yticks(np.arange(0, 1.1, 0.2))

axs[0].set_ylabel("Percentage")

for (
    ax,
    name,
) in zip(axs, names):
    frame = name2frame[name]
    plot_rank_distribution_cumul(
        frame,
        score_col="bsim_rank",
        ax=ax,
    )

    plot_rank_distribution_cumul(
        frame,
        score_col="neighbsim_rank",
        ax=ax,
    )

    ax.set_xlabel("Rank")


for (
    ax,
    name,
) in zip(axs, names):
    frame = name2frame[name]
    plot_rank_distribution(
        frame,
        score_col="bsim_rank",
        ax=ax,
    )

    plot_rank_distribution(
        frame,
        score_col="neighbsim_rank",
        ax=ax,
    )

    ax.set_title(
        name2label[name],
    )

legend = axs[-1].legend(
    loc="upper left",
    bbox_to_anchor=(1.02, 1),
)

score_col2text = {
    "bsim_rank": "BSim",
    "neighbsim_rank": "NeighBSim",
}
for text in legend.get_texts():
    text.set_text(score_col2text[text.get_text()])

In [ ]:
fig.savefig(
    "figures/ranking:architecture.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
names = [
    "mrr:noinlineXinline",
    "mrr:noltoXlto",
    "mrr:nopieXpie",
    "mrr:randomXrandom",
]

fig, axs = plt.subplots(
    nrows=1,
    ncols=len(names),
    figsize=(3 * len(names), 2.5),
    sharex=True,
    sharey=True,
)


max_rank = 10


for ax in axs:
    ax.set_xlim((0, max_rank))
    ax.set_ylim(0, 1)

    ax.set_xticks(np.arange(1, 11))
    ax.set_yticks(np.arange(0, 1.1, 0.2))

axs[0].set_ylabel("Percentage")

for (
    ax,
    name,
) in zip(axs, names):
    frame = name2frame[name]
    plot_rank_distribution_cumul(
        frame,
        score_col="bsim_rank",
        ax=ax,
    )

    plot_rank_distribution_cumul(
        frame,
        score_col="neighbsim_rank",
        ax=ax,
    )

    ax.set_xlabel("Rank")


for (
    ax,
    name,
) in zip(axs, names):
    frame = name2frame[name]
    plot_rank_distribution(
        frame,
        score_col="bsim_rank",
        ax=ax,
    )

    plot_rank_distribution(
        frame,
        score_col="neighbsim_rank",
        ax=ax,
    )

    ax.set_title(
        name2label[name],
    )

legend = axs[-1].legend(
    loc="upper left",
    bbox_to_anchor=(1.02, 1),
)

score_col2text = {
    "bsim_rank": "BSim",
    "neighbsim_rank": "NeighBSim",
}
for text in legend.get_texts():
    text.set_text(score_col2text[text.get_text()])

In [ ]:
fig.savefig(
    "figures/ranking:misc.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
# Bsim failed pretty hard in comparing size optimization and no optimization
frame = name2frame["mrr:osXo0"]
frame[frame["bsim_rank"] == 1]

In [ ]:
# How many functions from bsim rank two moved up by one in neighbsim
frame = name2frame["mrr:o0Xo2"]
len(frame[(frame["bsim_rank"] == 2) & (frame["neighbsim_rank"] == 1)]) / len(
    frame[frame["bsim_rank"] == 2]
)

In [ ]:
name2frame["mrr:o0Xo2"]["bsim_rank"].max()

### Comparison to FirmUP

In [ ]:
from evaluatie.firmup import firmup, FirmUPArgs
from evaluatie import utils
from evaluatie import models as m
import networkx as nx
import tqdm

tqdm.tqdm.pandas()

In [ ]:
plot_df = pd.DataFrame(
    columns=list(name2frame),
    index=["neighbsim", "bsim", "firmup"],
)

In [ ]:
for name, frame in name2frame.items():
    firmup_percentage = (frame["firmup"] == frame["ptarget_function_id"]).sum() / len(frame)
    bsim_percentage = (frame["bsim_rank"] == 1).sum() / len(frame)
    neighbsim_percentage = (frame["neighbsim_rank"] == 1).sum() / len(frame)

    plot_df.loc["firmup", name] = firmup_percentage
    plot_df.loc["bsim", name] = bsim_percentage
    plot_df.loc["neighbsim", name] = neighbsim_percentage

In [ ]:
plot_df

In [ ]:
plot_df = plot_df[
    [
        "mrr:o0Xo2",
        "mrr:o0Xo3",
        "mrr:osXo0",
        "mrr:osXo2",
        "mrr:osXo3",
        "mrr:x86Xarm",
        "mrr:x86Xmips",
        "mrr:armXmips",
        "mrr:noinlineXinline",
        "mrr:noltoXlto",
        "mrr:nopieXpie",
        "mrr:randomXrandom",
    ]
]

In [ ]:
plot_df.loc["firmup"] - plot_df.loc["neighbsim"]

In [ ]:
def as_latex_percent(value: float):
    percent = value * 100
    return f"${percent:.2f}\\%$"

In [ ]:
def latex_line_from_row(row):
    return " & ".join([as_latex_percent(value) for value in row])

In [ ]:
x = plot_df.apply(
    latex_line_from_row,
    axis=1,
)

In [ ]:
print(x.firmup)

In [ ]:
print(x.bsim)

In [ ]:
print(x.neighbsim)

## Ghidra Performance

In [ ]:
import pandas as pd
import pathlib as pl
import re

In [ ]:
def date_from_line(line: str):
    m = re.match(r"\[(.+?)\: .+?\]", line)
    assert m is not None

    date_str = m.group(1)

    return pd.to_datetime(date_str)

In [ ]:
def worker_from_line(line: str):
    m = re.match(r".+ForkPoolWorker-(\d+)\]", line)
    if m is None:
        return None

    worker_str = m.group(1)
    return int(worker_str)

In [ ]:
lines = pl.Path("./celery.stderr").read_text().splitlines()

In [ ]:
date_from_line(lines[0])

In [ ]:
worker_from_line(lines[100])

## Uncategorized

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LogNorm

In [ ]:
df = pd.read_csv(
    "datasets/factors:raw.csv.gz",
)

In [ ]:
df

In [ ]:
df["size"].quantile(q=0.95)

In [ ]:
df["neighborhood_size"].quantile(q=0.95)

In [ ]:
data = (
    df[
        [
            "size",
            "neighborhood_size",
        ]
    ][
        (df["size"] <= df["size"].quantile(q=0.95))
        & (df["neighborhood_size"] <= df["neighborhood_size"].quantile(q=0.95))
    ]
    .value_counts()
    .unstack()
)

In [ ]:
sns.heatmap(
    data,
    norm=LogNorm(),
)